# Setup API Key

In [1]:
import openai
import os
import dotenv
from dotenv import main


main.load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# Create Function

In [2]:
def get_response(prompt):
    # create a request to the chat completion endpoint
    response = openai.ChatCompletion.create(
        model = "gpt-3.5-turbo",
        messages = [{"role":"user", "content":prompt}],
        temperature = 0
    )
    return response.choices[0].message["content"]

# Chain-of-thought prompting
- Requires LLMs to provide reasoning steps(thoughts) before giving answer
- Solve the following problem step by step
- Used for complex reasoning tasks
- Help reduce model errors

## Standard prompting to solve a reasoning task

In [3]:
prompt = """Q: You start with 15 books in your collection. At the bookstore,
you purchase 8 new books. then, you lend 3 to your friend and 2 to your cousin.
Later, you visit another bookstore and buy 5 more books. How many books do
you have now?
A: The answer is: 
"""
print(get_response(prompt))

15 + 8 - 3 - 2 + 5 = 23 books.


## Chain-of-thought prompting to solve a reasoning task

In [4]:
prompt = """Q: You start with 15 books in your collection. At
the bookstore, you purchase 8 new books. Then, you lend 3 to your
friend and 2 to your cousin. Later, you visit another bookstore 
and buy 5 more books. How many books do you have now?
A: Let's think step by step
"""
print(get_response(prompt))

Step 1: Start with 15 books
Step 2: Purchase 8 new books
Total books = 15 + 8 = 23 books
Step 3: Lend 3 books to your friend
Total books = 23 - 3 = 20 books
Step 4: Lend 2 books to your cousin
Total books = 20 - 2 = 18 books
Step 5: Buy 5 more books
Total books = 18 + 5 = 23 books
So, you have 23 books now.


## Chain-of-thought prompting with few-shots

In [5]:
example = """
Q: The odd numbers in this group add up to an even number: 9, 10, 13, 4, 2.
A: Adding all the odd numbers (9, 13) gives 22. The answer is True.
"""
question = """
Q: The odd numbers in this group add up to an even number: 15, 13, 82, 7.
A: 
"""
prompt = example + question
print(get_response(prompt))

Adding all the odd numbers (15, 13, 7) gives 35. The answer is False.


# Chain-of-thought versus multi-step prompting

Multi-step prompts
- Incorporate steps inside the prompt

Chain-of-thought
- Ask model to generate intermediate steps

Limitation of chain-of-thought
- One unsuccessful thought --> unsuccessful answer

Solution to limitation
- Self-consistency prompts

# Self-consistency prompts
- generates multiple chain-of-thoughts by prompting the model several times
- majority vote to obtain final output

#### Can be done maually or through special prompt

In [6]:
self_consistency_instruction = """Imagine three completely independent experts who
reason differently are answering this question. The final answer
is obtained by majority vote. The question is:
"""
problem_to_solve = """If there are 10 cars in the parking lot and 
3 more cars arrive. Half the original number of cars leave. then, half of
the current number of cars arrive. How many cars are there in the 
parking?"""

prompt = self_consistency_instruction + problem_to_solve
print(get_response(prompt))

Expert 1: According to the given information, there are initially 10 cars in the parking lot. If 3 more cars arrive, the total number of cars becomes 10 + 3 = 13. Then, half of the original number of cars leave, which is 10 / 2 = 5 cars. Now, if half of the current number of cars arrive, it would be 13 / 2 = 6.5 cars. Since we cannot have half a car, we need to round it down to the nearest whole number. Therefore, the final answer is 5 + 6 = 11 cars in the parking lot.

Expert 2: Initially, there are 10 cars in the parking lot. If 3 more cars arrive, the total number of cars becomes 10 + 3 = 13. Then, half of the original number of cars leave, which is 10 / 2 = 5 cars. Now, if half of the current number of cars arrive, it would be 13 / 2 = 6.5 cars. Since we cannot have half a car, we need to round it up to the nearest whole number. Therefore, the final answer is 5 + 7 = 12 cars in the parking lot.

Expert 3: Initially, there are 10 cars in the parking lot. If 3 more cars arrive, the t

# Reasoning with chain-of-thought prompts
Chain-of-thought prompting is helpful to explain the reasoning behind the answer that the model is giving, especially in complex tasks such as generating the solution for a mathematical problem or a riddle. In this exercise, you will craft a chain-of-thought prompt to let the language model guess the age of your friend's father based on some information you will provide.

Create a chain-of-thought prompt to get the age of your friend's father in 10 years, given that now he's double your friend's age, and your friend is 20

In [8]:
prompt = """Determine the age of my friend's father in 10 years,
given that now he's double my friend's age and my friend is 20.
Let's think step by step
"""
response = get_response(prompt)
print(response)

Step 1: Determine your friend's current age.
Your friend is currently 20 years old.

Step 2: Determine your friend's father's current age.
Since your friend's father is currently double your friend's age, his current age is 20 * 2 = 40 years old.

Step 3: Determine your friend's father's age in 10 years.
In 10 years, your friend's father will be 40 + 10 = 50 years old.


# One-shot chain-of-thought prompts
When you need to sum the even numbers within a given set, you first have to identify these even numbers and then sum them. You can teach this to a language model via one or more examples, and it will follow this strategy to operate on new sets.

Your goal in this exercise is to teach the model how to apply this procedure on the following set: {9, 10, 13, 4, 2}, and then ask the model to perform it on a new set: {15, 13, 82, 7, 14}. This is how you perform chain-of-thought prompting through one-shot prompting.

- Define an example that teaches the model how to reason on the set {9, 10, 13, 4, 2}.
- Define a question, similar to the one in the example, that asks the model to reason on a new set {15, 13, 82, 7, 14}.
- Create the final prompt.

In [9]:
# Define the example 
example = """Q: Sum the even numbers in the following set: {9, 10, 13, 4, 2}.
             A: Even numbers: . Adding them: 10+4+2 =16"""

# Define the question
question = """Q: Sum the even numbers in the following set: {15, 13, 82, 7, 14}. 
              A:"""

# Create the final prompt
prompt = example + question
response = get_response(prompt)
print(response)

Even numbers: 82, 14. Adding them: 82 + 14 = 96


# Self-consistency prompts
Imagine you own a store that sells laptops and mobile phones. You start your day with 50 devices in the store, out of which 60% are mobile phones. Throughout the day, three clients visited the store, each of them bought one mobile phone, and one of them bought additionally a laptop. Also, you added to your collection 10 laptops and 5 mobile phones. How many laptops and mobile phones do you have by the end of the day? This problem is defined in the problem_to_solve string, and you will use a self-consistency prompt to solve it.
- Create the self_consistency_instruction that allows the model to solve the problem with three experts and combine the results with a majority vote.
- Create the final prompt by combining the self_consistency_instruction and the problem_to_solve.

In [10]:
# Create the self_consistency instruction
self_consistency_instruction = """Imagine three independent experts
who think differently are answering this question. 
The final answer is obtained by majority vote."""

# Create the problem to solve
problem_to_solve = """If you own a store that sells laptops and 
mobile phones. You start your day with 50 devices in the store, 
out of which 60% are mobile phones. 
Throughout the day, three clients visited the store, 
each of them bought one mobile phone, and 
one of them bought additionally a laptop. 
Also, you added to your collection 10 laptops and 5 mobile phones. 
How many laptops and mobile phones do you have by the end of the 
day?"""

# Create the final prompt
prompt = self_consistency_instruction + problem_to_solve

response = get_response(prompt)
print(response)

Expert 1: 

At the start of the day, we have 60% of 50 devices as mobile phones, which is 30 mobile phones. The remaining 40% are laptops, which is 20 laptops. 

Throughout the day, three clients bought one mobile phone each, so we subtract 3 mobile phones from our count. One of the clients also bought a laptop, so we subtract one laptop from our count. 

We added 10 laptops and 5 mobile phones to our collection. 

Therefore, at the end of the day, we have:
Mobile phones: 30 - 3 + 5 = 32 mobile phones
Laptops: 20 - 1 + 10 = 29 laptops

Expert 2: 

At the start of the day, we have 60% of 50 devices as mobile phones, which is 30 mobile phones. The remaining 40% are laptops, which is 20 laptops. 

Throughout the day, three clients bought one mobile phone each, so we subtract 3 mobile phones from our count. One of the clients also bought a laptop, so we subtract one laptop from our count. 

We added 10 laptops and 5 mobile phones to our collection. 

Therefore, at the end of the day, we ha